# Houston, TX Simulations
8760 rows per simulation<br>
50 files (removed cell-in-cell)

In [1]:
# Dependencies
import pandas as pd
import os
import numpy as np
import pprint
from pathlib import Path
from datetime import datetime

## Energy Conversions

As per E* Documentation<br>

Joules --> kBtu (multiply by .00000094781712)<br>
Joules --> kWh (multiply by .000000277777778) or (divide by 3600000)<br>
Joules --> therms (multiply by .0000000094781712<br>

foot-candles --> Lux (multiply by 10.7642)

<div>
    <img src="static/images/elect_conv.png" width="400"/>
     <img src="static/images/gas_conv.png" width="400"/>
</div

In [2]:
# 2018 Egrid Costs (elec $/kWh, gas $/therm), change values per city
gas = 1.11
elec = 0.112

---

## Combine CSV Files in Directory

In [4]:
# Create variable for files in directory
files = [f for f in os.listdir("data/hou/") if f.endswith(".csv")]

# files

In [5]:
# https://stackoverflow.com/questions/63886787/how-to-create-a-dataframe-from-multiple-csv-files
# Initialize and empty dataframe
all_data = pd.DataFrame()

# Iterate through files and contents, then concatenate the data into the dataframe
for file in files:
    df = pd.read_csv("data/hou/" + file)
    all_data = pd.concat([all_data, df])
    
#all_data

# 438000  rows, 73 columns

In [6]:
# all_data.dtypes

In [7]:
# print(all_data)
# is_NaN = all_data. isnull()
# row_has_NaN = is_NaN. any(axis=1)
# rows_with_NaN = all_data[row_has_NaN]
# print(rows_with_NaN)

In [8]:
# Get wanted columns
all_data1 = all_data[["Scenario",
               "Date/Time",
              "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
              "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)",
              "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)",
              "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) "]]

#all_data1

In [9]:
all_data1 = all_data1.dropna().reset_index(drop=True)

#all_data1

#438000  rows, 73 columns

-----

## Get the Wanted Data (columns)

In [10]:
# Rename Columns
all_data2 = all_data1.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#all_data2

#438000  rows, 6 columns

-----

## Get Annual Energy Data
Convert units from Joules to kBtu<br>
Drop Illuminance column<br>


In [11]:
# Copy dataframe to modify and leave original df intact
get_annual = all_data2.copy(deep=True)

#get_annual

In [12]:
# Convert Joules to KBtu in dataframe
get_annual["FanEnergy(kBtu)"] = get_annual["FanEnergy[J](Hourly)"] * .00000094781712
get_annual["HeatingEnergy(kBtu)"] = get_annual["HeatingEnergy[J](Hourly)"] * .00000094781712
get_annual["CoolingEnergy(kBtu)"] = get_annual["CoolingEnergy[J](Hourly) "] * .00000094781712

#get_annual

In [13]:
# Drop columns
get_annual.drop(columns = ["Illuminance(lux)", "FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_annual

---

## Get Annual Cost Data
Convert units from Joules to kBtu<br>
Drop Illuminance column<br>
Add cost multiplier, variables defined above

In [14]:
# Get annual costs
get_annual_costs = all_data2.copy(deep=True)

In [15]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_annual_costs["FanEnergy($)"] = (get_annual_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_annual_costs["HeatingEnergy($)"] = ((get_annual_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_annual_costs["CoolingEnergy($)"] = (get_annual_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_annual_costs

In [16]:
# Drop columns
get_annual_costs.drop(columns = ["Illuminance(lux)", "FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_annual_costs

---

## Combine Annual Energy and Cost Data
Combine two dfs<br>
Add baseline data <br>
Export combine file

In [17]:
# Combine data for energy and cost
allAnnualData = pd.merge(get_annual, get_annual_costs)

#allAnnualData

In [18]:
# Add energy and cost totals
totalEnergy = allAnnualData["FanEnergy(kBtu)"] + allAnnualData["HeatingEnergy(kBtu)"] + allAnnualData["CoolingEnergy(kBtu)"]
allAnnualData["AnnualEnergy(kBtu)"] = totalEnergy

totalCost = allAnnualData["FanEnergy($)"] + allAnnualData["HeatingEnergy($)"] + allAnnualData["CoolingEnergy($)"]
allAnnualData["AnnualCost($)"] = totalCost

#allAnnualData


### Establish Annual Energy and Cost Baseline Data


In [19]:
# Establish energy target. % improvement from the average of Heat + Cool + Fan of Clear/Clear-VB0 (dual clear with a Venetian Blind at 0 degrees).
baseline_annual = allAnnualData[allAnnualData["Scenario"] == "Clear/Clear-VB0"]

#baseline_annual

In [20]:
annualEnergy_fan = baseline_annual["FanEnergy(kBtu)"].sum()
annualEnergy_fan

5535.829631227591

In [21]:
annualEnergy_cool = baseline_annual["CoolingEnergy(kBtu)"].sum()
annualEnergy_cool

31170.891245358252

In [22]:
annualEnergy_heat = baseline_annual["HeatingEnergy(kBtu)"].sum()
annualEnergy_heat

19289.426003497585

In [23]:
annualEnergy = (annualEnergy_fan + annualEnergy_cool + annualEnergy_heat)
annualEnergy

55996.14688008343

In [24]:
annualCost_fan = baseline_annual["FanEnergy($)"].sum()
annualCost_fan

181.70784980870937

In [25]:
annualCost_cool = baseline_annual["CoolingEnergy($)"].sum()
annualCost_cool

1023.1520841726409

In [26]:
annualCost_heat = baseline_annual["HeatingEnergy($)"].sum()
annualCost_heat

214.11262863882322

In [27]:
annualCost = (annualCost_fan + annualCost_cool + annualCost_heat)
annualCost

1418.9725626201734

---
## Group and Export Annual Data
Group dataframe by scenario<br>
Add baselines to dataframe<br>
Export to file

In [28]:
# Group by scenario
annualData = allAnnualData.groupby(["Scenario"], as_index=False)

#annualData.sum().round(2)

In [29]:
annualDataFinal = annualData.sum().round(2)

#annualDataFinal

In [30]:
# Add columns for delta from base
annualDataFinal["DeltaEnergyFromBase"] = annualDataFinal["AnnualEnergy(kBtu)"] - annualEnergy
annualDataFinal["DeltaCostFromBase"] = annualDataFinal["AnnualCost($)"] - annualCost

#annualDataFinal.round(2)

In [31]:
# Export grouped file to csv
annualDataFinal.to_csv("../Scraper_Output/hou_annual_data.csv", header=True, index=True)

-----

## Get Monthly Energy Data
Change units from Joules to therms and kWh<br>
Drop unwanted columns

In [32]:
# Copy dataframe to modify and leave original df intact
get_monthly = all_data2.copy(deep=True)

#get_monthly

In [33]:
# Convert Joules to kWh and therms in new dataframe
get_monthly["FanEnergy(kWh)"] = get_monthly["FanEnergy[J](Hourly)"] * .000000277777778
get_monthly["HeatingEnergy(therms)"] = get_monthly["HeatingEnergy[J](Hourly)"] * .0000000094781712
get_monthly["CoolingEnergy(kWh)"] = get_monthly["CoolingEnergy[J](Hourly) "] * .000000277777778

#get_monthly

In [34]:
# Drop columns
get_monthly.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_monthly

In [35]:
# get_monthly["Month"] = get_monthly["Date_Time"].apply(lambda x: str(x)[0:3])

# get_monthly

## Get Monthly Cost Data

Change units from Joules to therms and kWh<br>
Add cost multiplier, variable needs to be defined above

In [36]:
# Copy dataframe to modify and leave original df intact
get_monthly_costs = all_data2.copy(deep=True)

#get_monthly

In [37]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_monthly_costs["FanEnergy($)"] = (get_monthly_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_monthly_costs["HeatingEnergy($)"] = ((get_monthly_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_monthly_costs["CoolingEnergy($)"] = (get_monthly_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_monthly_costs

In [38]:
# Drop columns
get_monthly_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_monthly_costs

In [39]:
# get_monthly_costs["Month"] = get_monthly_costs["Date_Time"].apply(lambda x: str(x)[0:3])

# get_monthly_costs

---

## Combine Monthly Energy and Cost Data
Combine two dfs<br>
Create Month Column<br>
Establish baseline <br>
Export combine file

In [40]:
# Combine data for energy and cost
allMonthlyData = pd.merge(get_monthly, get_monthly_costs, how="inner" )

#allMonthlyData

In [41]:
# Add energy and cost totals
# totalEnergyMonthly = allMonthlyData["FanEnergy(kWh)"] + allMonthlyData["HeatingEnergy(therms)"] + allMonthlyData["CoolingEnergy(kWh)"]
# allMonthlyData["MonthlyEnergy(kBtu)"] = totalEnergyMonthly

totalCostMonthly = allMonthlyData["FanEnergy($)"] + allMonthlyData["HeatingEnergy($)"] + allMonthlyData["CoolingEnergy($)"]
allMonthlyData["MonthlyCost($)"] = totalCostMonthly

averageIlluminanceMonthly = allMonthlyData["Illuminance(lux)"].mean()

#allMonthlyData

In [42]:
allMonthlyData["Month"] = allMonthlyData.Date_Time.str.slice(0,3)

#allMonthlyData

In [43]:
# Drop Illuminance column...don't need the totals
allMonthlyData2 = allMonthlyData.drop(["Illuminance(lux)"], axis=1)
#allMonthlyData2


### Establish Monthly Energy and Cost Baseline Data


In [44]:
# Establish energy target. % improvement from the average of Heat + Cool + Fan of Clear/Clear-VB0 (dual clear with a Venetian Blind at 0 degrees).
baseline_monthly = allMonthlyData2[allMonthlyData2["Scenario"] == "Clear/Clear-VB0"]

#baseline_monthly

In [45]:
monthlyEnergy_fan = baseline_monthly["FanEnergy(kWh)"].sum()
monthlyEnergy_fan

1622.3915161491905

In [46]:
monthlyEnergy_cool = baseline_monthly["CoolingEnergy(kWh)"].sum()
monthlyEnergy_cool

9135.286465827152

In [47]:
monthlyEnergy_heat = baseline_monthly["HeatingEnergy(therms)"].sum()
monthlyEnergy_heat

192.89426003497584

In [48]:
monthlyCost_fan = baseline_monthly["FanEnergy($)"].sum()
monthlyCost_fan

181.70784980870937

In [49]:
monthlyCost_cool = baseline_monthly["CoolingEnergy($)"].sum()
monthlyCost_cool

1023.1520841726409

In [50]:
monthlyCost_heat = baseline_monthly["HeatingEnergy($)"].sum()
monthlyCost_heat

214.11262863882322

In [51]:
monthlyCost = (monthlyCost_fan + monthlyCost_cool + monthlyCost_heat)
monthlyCost

1418.9725626201734

## Group and Export Monthly Data
Group dataframe by scenario and month<br>
Add baselines to dataframe<br>
Export to file

In [52]:
# Group by scenario and date/time for export to graphing
MonthlyData = allMonthlyData2.groupby(["Scenario", "Month"], as_index=False)

#MonthlyData.sum()

In [53]:
MonthlyDataFinal = MonthlyData.sum()

#MonthlyDataFinal

In [54]:
# Add columns for delta from base
MonthlyDataFinal["DeltaFanFromBase"] = MonthlyDataFinal["FanEnergy(kWh)"] - monthlyEnergy_fan
MonthlyDataFinal["DeltaCoolFromBase"] = MonthlyDataFinal["CoolingEnergy(kWh)"] - monthlyEnergy_cool
MonthlyDataFinal["DeltaHeatFromBase"] = MonthlyDataFinal["HeatingEnergy(therms)"] - monthlyEnergy_heat
MonthlyDataFinal["DeltaCostFromBase"] = MonthlyDataFinal["MonthlyCost($)"] - monthlyCost

#MonthlyDataFinal.round(2)

In [55]:
# Export grouped file to csv (used before grouped by month)
MonthlyDataFinal.round(2).to_csv("../Scraper_Output/hou_monthly_data.csv", header=True, index=True)

In [56]:
# # Group by scenario and date/time for export to graphing
# MonthlyData2Final = allMonthlyData2.groupby(["Scenario", "Month"], as_index=False)

# MonthlyData2Final.sum()

In [57]:
# # Export grouped file to csv
# MonthlyData2Final.sum().round(2).to_csv("../Scraper_Output/hou_monthly2_data.csv", header=True, index=True)

## Get Monthly Average Illuminance
Get just the illuminance column<br>
Group by scenario <br>
Groups by month <br>

In [58]:
Illuminance = allMonthlyData[["Scenario", "Date_Time", "Illuminance(lux)", "Month"]]

#Illuminance

In [59]:
# Group by scenario
MonthlyIlluminance = Illuminance.groupby(["Scenario", "Month"])

#MonthlyIlluminance.mean()

In [60]:
# Export grouped file to csv
MonthlyIlluminance.mean().round(2).to_csv("../Scraper_Output/hou_monthly_illuminance.csv", header=True, index=True)

------

## Get Hourly Energy Data
Change units from Joules to therms and kWh<br>
Drop unwanted columns

In [61]:
# Copy dataframe to modify and leave original df intact
all_data3 = all_data1.copy(deep=True)

#all_data3

In [62]:
# Get wanted columns
# Include hourly illumance
all_data3 = all_data3[["Scenario",
               "Date/Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) "
                      ]]

#all_data3

In [63]:
# Rename Columns
get_hourly = all_data3.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

#get_hourly

In [64]:
# Convert Joules to kWh and therms in new dataframe
get_hourly["FanEnergy(kWh)"] = get_hourly["FanEnergy[J](Hourly)"] * .000000277777778
get_hourly["HeatingEnergy(therms)"] = get_hourly["HeatingEnergy[J](Hourly)"] * .0000000094781712
get_hourly["CoolingEnergy(kWh)"] = get_hourly["CoolingEnergy[J](Hourly) "] * .000000277777778

#get_hourly

In [65]:
# Drop columns
get_hourly.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_monthly

------

## Get Hourly Cost Data
Change units from Joules to therms and kWh<br>
Add cost multiplier, variable needs to be defined above

In [66]:
# Copy dataframe to modify and leave original df intact
get_hourly_costs = all_data3.copy(deep=True)

#get_hourly_costs

In [67]:
# Rename Columns
get_hourly_costs = get_hourly_costs.rename(columns={"Scenario":"Scenario",
                              "Date/Time":"Date_Time",
                             "LIVING_UNIT1 DAYLIGHTINGCONTROLS:Daylighting Reference Point 1 Illuminance [lux](Hourly)":"Illuminance(lux)",
                             "SUPPLY FAN_UNIT1:Fan Electricity Energy [J](Hourly)":"FanEnergy[J](Hourly)",
                             "MAIN GAS HEATING COIL_UNIT1:Heating Coil NaturalGas Energy [J](Hourly)":"HeatingEnergy[J](Hourly)",
                             "DX COOLING COIL_UNIT1:Cooling Coil Electricity Energy [J](Hourly) ":"CoolingEnergy[J](Hourly) "
                             })

# get_hourly_costs

In [68]:
# Convert Joules to KBtu in dataframe and add cost multiplier
# Gas kbtu => Mbtu => therms: (kbtu/1000) * 10
# Elec kbtu => Kwh (kbtu * .000000277777778)
get_hourly_costs["FanEnergy($)"] = (get_hourly_costs["FanEnergy[J](Hourly)"] * .000000277777778) * elec
get_hourly_costs["HeatingEnergy($)"] = ((get_hourly_costs["HeatingEnergy[J](Hourly)"] * .00000094781712)/1000) * 10  * gas
get_hourly_costs["CoolingEnergy($)"] = (get_hourly_costs["CoolingEnergy[J](Hourly) "] * .000000277777778) * elec

#get_monthly_costs

In [69]:
# Drop columns
get_hourly_costs.drop(columns = ["FanEnergy[J](Hourly)", "HeatingEnergy[J](Hourly)", "CoolingEnergy[J](Hourly) "], axis=1, inplace=True)

#get_hourly_costs

## Combine Hourly Energy and Cost Data
Combine two dfs<br>
Establish baseline <br>
Export combine file

In [70]:
# Combine data for energy and cost
allHourlyData = pd.merge(get_hourly, get_hourly_costs, how="inner" )

#allHourlyData

In [71]:
# Add energy cost totals

totalCosthourly = allHourlyData["FanEnergy($)"] + allHourlyData["HeatingEnergy($)"] + allHourlyData["CoolingEnergy($)"]
allHourlyData["HourlyCost($)"] = totalCosthourly

#allHourlyData


### Establish Hourly Energy and Cost Baseline Data


In [72]:
# Establish energy target. % improvement from the average of Heat + Cool + Fan of Clear/Clear-VB0 (dual clear with a Venetian Blind at 0 degrees).
baseline_hourly = allHourlyData[allHourlyData["Scenario"] == "Clear/Clear-VB0"]

#baseline_hourly

In [73]:
hourlyEnergy_fan = baseline_hourly["FanEnergy(kWh)"].mean()
hourlyEnergy_fan

0.18520451097593474

In [74]:
hourlyEnergy_cool = baseline_hourly["CoolingEnergy(kWh)"].mean()
hourlyEnergy_cool

1.0428409207565226

In [75]:
hourlyEnergy_heat = baseline_hourly["HeatingEnergy(therms)"].mean()
hourlyEnergy_heat

0.022019892698056626

In [76]:
hourlyCost_fan = baseline_hourly["FanEnergy($)"].mean()
hourlyCost_fan

0.02074290522930468

In [77]:
hourlyCost_cool = baseline_hourly["CoolingEnergy($)"].mean()
hourlyCost_cool

0.11679818312473092

In [78]:
hourlyCost_heat = baseline_hourly["HeatingEnergy($)"].mean()
hourlyCost_heat

0.024442080894842858

In [79]:
hourlyCost = (hourlyCost_fan + hourlyCost_cool + hourlyCost_heat)
hourlyCost

0.16198316924887846

## Export Hourly Data

Add baselines to dataframe<br>
Export to file

In [80]:
# Add columns for delta from base
allHourlyData["DeltaFanFromBase"] = allHourlyData["FanEnergy(kWh)"] - hourlyEnergy_fan
allHourlyData["DeltaCoolFromBase"] = allHourlyData["CoolingEnergy(kWh)"] - hourlyEnergy_cool
allHourlyData["DeltaHeatFromBase"] = allHourlyData["HeatingEnergy(therms)"] - hourlyEnergy_heat
allHourlyData["DeltaCostFromBase"] = allHourlyData["HourlyCost($)"] - hourlyCost

allHourlyData.round(2)

,Scenario,Date_Time,Illuminance(lux),FanEnergy(kWh),HeatingEnergy(therms),CoolingEnergy(kWh),FanEnergy($),HeatingEnergy($),CoolingEnergy($),HourlyCost($),DeltaFanFromBase,DeltaCoolFromBase,DeltaHeatFromBase,DeltaCostFromBase
0,Clear/Clear,01/01 01:00:00,0.0,0.17,0.18,0.0,0.02,0.20,0.0,0.22,-0.02,-1.04,0.15,0.05
1,Clear/Clear,01/01 02:00:00,0.0,0.19,0.19,0.0,0.02,0.21,0.0,0.23,0.00,-1.04,0.17,0.07
2,Clear/Clear,01/01 03:00:00,0.0,0.20,0.21,0.0,0.02,0.23,0.0,0.25,0.01,-1.04,0.18,0.09
3,Clear/Clear,01/01 04:00:00,0.0,0.21,0.22,0.0,0.02,0.24,0.0,0.27,0.03,-1.04,0.20,0.11
4,Clear/Clear,01/01 05:00:00,0.0,0.22,0.23,0.0,0.02,0.25,0.0,0.28,0.03,-1.04,0.20,0.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437995,lowE(2)/Clear-BBG75,12/31 20:00:00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,-0.19,-1.04,-0.02,-0.16
437996,lowE(2)/Clear-BBG75,12/31 21:00:00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,-0.19,-1.04,-0.02,-0.16
437997,lowE(2)/Clear-BBG75,12/31 22:00:00,0.0,0.00,0.00,0.0,0.00,0.00,0.0,0.00,-0.18,-1.04,-0.02,-0.16
437998,lowE(2)/Clear-BBG75,12/31 23:00:00,0.0,0.04,0.04,0.0,0.00,0.05,0.0,0.05,-0.14,-1.04,0.02,-0.11


In [81]:
# Export grouped file to csv for daily cost sums
allHourlyData.to_csv("../Scraper_Output/hou_daily.csv", header=True, index=False)